This is an accompanying notebook for the paper *AKLT-states as ZX-diagrams: diagrammatic reasoning for quantum states*, by East, van de Wetering, Chancellor and Grushin.

In this notebook we demonstrate how a 2D AKLT state on a hexagonal lattice can be constructed and simplified inside PyZX.

In [ ]:
import sys; sys.path.insert(0,'../..')
import random
import pyzx as zx # This notebook was created with pyzx version 0.6.3 in mind, newer version might be incompatible.
from fractions import Fraction
zx.settings.drawing_auto_hbox = False

First, we construct the spin-3/2 symmetriser, that projects the 8-dimensional subspace of three qubit wires down to the 4-dimensional symmetric subspace.

In [ ]:
# We load the diagram via its description as a tikz diagram
tikz_symmetrizer = r"""\begin{tikzpicture}[tikzfig]
	\begin{pgfonlayer}{nodelayer}
		\node [style=Red Node] (76) at (0.75, -0.75) {};
		\node [style=Red Node] (77) at (3, 0) {};
		\node [style=Red Node] (78) at (3, -1.5) {};
		\node [style=Red Node] (79) at (6.75, -3) {};
		\node [style=Red Node] (80) at (6.75, 0) {};
		\node [style=Red Node] (81) at (3.75, -2) {};
		\node [style=Green Node] (82) at (0.75, -1.5) {};
		\node [style=Green Node] (83) at (0.75, 0) {};
		\node [style=Green Node] (84) at (3, -0.75) {};
		\node [style=Green Node] (85) at (3.75, 0) {};
		\node [style=Green Node] (86) at (6.75, -2) {};
		\node [style=Green Node] (87) at (4.75, -0.25) {};
		\node [style=H] (88) at (4.75, -1) {};
		\node [style=none] (89) at (10.25, 0) {};
		\node [style=none] (90) at (0, 0) {};
		\node [style=none] (91) at (0, -1.5) {};
		\node [style=none] (92) at (0, -3) {};
		\node [style=Green Node] (93) at (3.75, -3) {};
		\node [style=Green Node] (94) at (1.5, -0.25) {};
		\node [style=H] (95) at (1.5, -0.75) {};
		\node [style=H] (96) at (4.75, -2) {};
		\node [style=H] (97) at (2.25, -0.75) {};
		\node [style=H] (98) at (5.75, -2) {};
		\node [style=Green Node] (99) at (6, -1) {};
		\node [style=Red Node] (100) at (7.75, -2.25) {};
		\node [style=Red Node] (101) at (9.75, -1.5) {};
		\node [style=Red Node] (102) at (9.75, -3) {};
		\node [style=Green Node] (103) at (7.75, -3) {};
		\node [style=Green Node] (104) at (7.75, -1.5) {};
		\node [style=Green Node] (105) at (9.75, -2.25) {};
		\node [style=H] (106) at (8.25, -2.25) {};
		\node [style=H] (107) at (9, -2.25) {};
		\node [style=none] (108) at (10.25, -1.5) {};
		\node [style=none] (109) at (10.25, -3) {};
	\end{pgfonlayer}
	\begin{pgfonlayer}{edgelayer}
		\draw (90.center) to (83);
		\draw (82) to (78);
		\draw (83) to (77);
		\draw (77) to (85);
		\draw (85) to (80);
		\draw (80) to (89.center);
		\draw (93) to (79);
		\draw (79) to (86);
		\draw (81) to (93);
		\draw (82) to (76);
		\draw (76) to (83);
		\draw (78) to (84);
		\draw (84) to (77);
		\draw (81) to (85);
		\draw (86) to (80);
		\draw (88) to (87);
		\draw (95) to (94);
		\draw (81) to (96);
		\draw (96) to (98);
		\draw (98) to (86);
		\draw (76) to (95);
		\draw (95) to (97);
		\draw (97) to (84);
		\draw (88) to (99);
		\draw (82) to (91.center);
		\draw (93) to (92.center);
		\draw (103) to (102);
		\draw (104) to (101);
		\draw (103) to (100);
		\draw (100) to (104);
		\draw (102) to (105);
		\draw (105) to (101);
		\draw (100) to (106);
		\draw (106) to (107);
		\draw (107) to (105);
		\draw (104) to (78);
		\draw (103) to (79);
		\draw (108.center) to (101);
		\draw (109.center) to (102);
		\draw (88) to (96);
		\draw [bend left=15, looseness=0.75] (99) to (106);
	\end{pgfonlayer}
\end{tikzpicture}
"""
symmetrizer = zx.Graph.from_tikz(tikz_symmetrizer)
symmetrizer.auto_detect_io() # Tell PyZX which boundary vertices are inputs and outputs
symmetrizer.scalar.add_power(-1) # Correct normalisation
zx.draw(symmetrizer)
zx.print_matrix(symmetrizer)

Now we construct the three POVM elements that we will use to measure the AKLT state.

In [ ]:
E_z = zx.generate.spider("Z",3,1)
E_x = zx.generate.spider("X",3,1)
S = zx.generate.spider("Z",1,1,phase=Fraction(1,2)) # The S gate
Sdg = S.adjoint() # Adjoint of S
E_y = (Sdg @ Sdg @ Sdg) + E_x + S # The symbol @ represents tensor product, while + is horizontal composition in diagrammatic order.
zx.draw(E_z @ E_x @ E_y)

Let us now demonstrate that each of these POVM elements applied to the symmetriser results in just the POVM element (up to a scalar factor):

In [ ]:
diagram = symmetrizer + E_z
zx.draw(diagram)
zx.hsimplify.zh_simp(diagram)
zx.draw(diagram)

Note that the resulting diagram has this 'floating' scalar subdiagram (you might need to move some spiders in the figure to see this clearly). This scalar diagram is equal to the number 6 and can be safely ignored.
While in the paper we used several applications of the bialgebra rule to simplify this diagram, PyZX uses a rule based on pivoting (described in Appendix B of the paper), that is more friendly to automation.

We can do the same thing with `E_x`, but here we get a bunch of blue wires that represent wires with a Hadamard on them. Hence, by using the 'colour-change' rule that relates the Z and X spider we get the desired result:

In [ ]:
diagram = symmetrizer + E_x
zx.hsimplify.zh_simp(diagram,quiet=True)
print("Simplified:")
zx.draw(diagram)
print("With the correct colour:")
zx.to_rg(diagram) # rg stands for 'red-green'
zx.draw(diagram)

Simplifying the diagram but with `E_y` instead of `E_x` results in a bit more complicated diagram. This is however very close to the  diagram that pyzx produces when applied to `E_y` directly. It is simply the 'fixed-point' that pyzx finds.

In [ ]:
print("First E_y:")
zx.draw(E_y)
print("E_y applied to the symmetriser and simplified:")
diagram = symmetrizer + E_y
zx.hsimplify.zh_simp(diagram,quiet=True)
zx.draw(diagram)
print("This looks more complicated, but note that 'simplifying' E_y directly gives basically the same result:")
g = E_y.copy()
zx.hsimplify.zh_simp(g,quiet=True)
zx.draw(g)
print("Indeed, these implement the same linear map up to global scalar:")
display(zx.print_matrix(diagram))
display(zx.print_matrix(g))

Now let us create one hexagonal cell consisting of the spin-3/2 symmetrisers connected by singlets.
PyZX doesn't allow us to easily organise the spin-3/2 components in a hexagonal pattern so the reader will have to some imagination.
In the cell below we first construct the six separate components with the $\pi$ phases required for the hexagonal cell.
The way these components should be interpreted is that we go from top left to bottom right. So the first 3 wires are the top left cell, the second 3 wires are the top right cell, then we go middle left, middle right, bottom left and bottom right.

In [ ]:
hexgrid = symmetrizer @ symmetrizer.translate(10,0) @ symmetrizer @ symmetrizer.translate(10,0) @ symmetrizer @ symmetrizer.translate(10,0)
Y = zx.generate.spider("Z",1,1,phase=1) + zx.generate.spider("X",1,1,phase=1)
ID = zx.generate.identity(1)
paulis = (Y @ Y @ Y @ Y.translate(0,1.3) @ ID @ Y @ ID.translate(0,1.3) @ Y @ Y @ ID.translate(0,1.3) 
          @ Y @ Y @ ID.translate(0,1.3) @ Y @ Y @ ID @ ID @ Y)
hexgrid =  paulis + hexgrid
zx.draw(hexgrid)

Now we connect the components together according to the hexagonal pattern.
The code might look a bit arbitrary, but with reference to Figure 4 of the paper one can see that we get the same structure.

In [ ]:
connectivity = [(1,4), (2,6), (5,9), (8,12), (11,15), (13,16)]
rem_verts = []
inputs = list(hexgrid.inputs())
for i,j in connectivity:
    b1 = inputs[i]
    b2 = inputs[j]
    v1 = list(hexgrid.neighbors(b1))[0] # works because b1 only has a single neighbor
    v2 = list(hexgrid.neighbors(b2))[0]
    hexgrid.add_edge(g.edge(v1,v2))
    rem_verts.append(b1)
    rem_verts.append(b2)

for b in rem_verts:
    inputs.remove(b)
hexgrid.set_inputs(tuple(inputs))
hexgrid.remove_vertices(rem_verts)
zx.draw(hexgrid)

Now we measure each of the spin-3/2s with one of the components of the POVM. Change the seed of the random generation to change the outcome.

In [ ]:
random.seed(458348)
measurement = zx.Graph()
for i in range(6):
    c = random.choice(['z','x','y'])
    if c == 'z': measurement = measurement @ E_z.translate(0,0.7)
    if c == 'x': measurement = measurement @ E_x.translate(0,0.7)
    if c == 'y': measurement = measurement @ E_y.translate(0,0.7)
g = hexgrid.copy()
g = g + measurement
zx.draw(g)

By simplifying the diagram we see that this reduces to a Clifford diagram.

In [ ]:
zx.hsimplify.zh_simp(g)
g.normalize() # Change the coordinates of the spiders for easier drawing
zx.draw(g)

Note that this is not a stabiliser state as we also have inputs. These inputs correspond to the open edges corresponding to the part of the hexagonal lattice we have not filled in. By picking some arbitrary Clifford inputs here we get a graph state up to some local Cliffords as expected.